In [1]:
import json

import networkx as nx

In [6]:
def get_imp(text):
    for i in range(10):
        text=text.split(str(i))[0]
    for i in ['یک', 'دو', 'سه', 'چهار']:
        text=text.split(i)[0]
    return text.split("به")[0].rstrip().lstrip()

In [7]:
with open('searches/data/all_foods.json', mode='r') as f:
    data = json.load(f)
for food in data:
    for i in range(len(food['ingredients'])):
        food['ingredients'][i]=get_imp(food['ingredients'][i])
foods = [i['name'].strip() for i in data]
ingredients_foods = {j.strip():[] for i in data for j in i['ingredients']}
for i in data:
    for j in i['ingredients']:
        ingredients_foods[j.strip()].append(i['name'])


<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
گراف موادغذایی-غذا
</div>

<div dir=rtl style="font-size: 18px; font-family: XB Niloofar;">
این گراف یک گراف دو بخشی است که یک بخش آن مواد غذایی و بخش دیگر آن غذاها هستند. بین یک غذا و ماده غذایی یال وجود دارد اگر که آن ماده غذایی در غذا به کار رفته باشد.
</div>

In [8]:
G = nx.Graph()
G.add_nodes_from(foods)
G.add_nodes_from(ingredients_foods.keys())

In [9]:
for i, j in ingredients_foods.items():
    for k in j:
        G.add_edge(i, k)

In [10]:
pr = nx.pagerank(G)
with open("searches/data/link1.json", "w", encoding="utf-8") as file:
    link1=sorted([(i, j) for i, j in pr.items()], key=lambda x: x[1], reverse=True)
    json.dump(link1, file)